## Example notebook component pipeline

First component:

*  get's `a`, `b`, `c` inputs from pipeline input.

* executes notebook `notebook_step_1.py` that uses those variables and ouputs variable `d` and artifact `x`

Second component:

* injests variable `d` and artifact `x` from previous step


In [1]:
from kfn import NbComponentBuilder
import kfp

In [ ]:
base_image = 'asia.gcr.io/ppb-services/tensorflow-1.14.0-py3-jupytext-papermill:latest'
packages_to_install = ['pandas', 'matplotlib']

In [2]:
nb = NbComponentBuilder(
    'Task 1', 
    inject_notebook_path='notebook_step_1.py', 
    remove_nb_inputs=True)

nb.add_input_param('a', int, 1)
nb.add_input_param('b', int, 2)
nb.add_input_param('c', int, None)
nb.add_output_param('d', int)
nb.add_output_artifact('x')
first_op = nb.build_op(
    base_image=base_image, 
    packages_to_install=packages_to_install)

SIGNATURE:  (c: int, a: int = '1', b: int = '2')


In [ ]:
nb = NbComponentBuilder(
    'Task 2', 
    inject_notebook_path='notebook_step_2.py')

nb.add_input_artifact('x')
nb.add_input_param('d', int)
second_op = nb.build_op(
    base_image=base_image, 
    packages_to_install=packages_to_install)

In [ ]:
# Define pipeline
@kfp.dsl.pipeline(name='Notebook pipeline example')
def notebook_run_pipeline(
    a='a', b='b', c='c'
):
    op1 = first_op(a=a, b=b, c=c)    
    second_op(x=op1.outputs['x'], d=op1.outputs['d'])    

# Run
arguments = {'a': 20, 'b': 30, 'c': 40}
kfp.Client().create_run_from_pipeline_func(notebook_run_pipeline, arguments=arguments)